In [47]:
import pandas as pd
import numpy as np

In [48]:
housing = pd.read_csv('housing_cleaned_1.csv')

In [49]:
#Splitting dataset into independent and dependent variables
X = housing.drop(['SalePrice'], axis=1)
y = housing['SalePrice']

In [50]:
# Splitting into training and test set
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X,y,test_size = 0.3, random_state = 42)

In [51]:
# feature names in a list
feature_names_train = [x for x in X_train.columns]
feature_names_test = [x for x in X_test.columns]

# Feature Scaling
from sklearn.preprocessing import MinMaxScaler
scaler=MinMaxScaler()
scaler.fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

# Convert back to dataframe
X_train = pd.DataFrame(data = X_train, columns = feature_names_train)
X_test = pd.DataFrame(data = X_test, columns = feature_names_test)

<strong>Use Lasso to select the more important features.</strong>

In [52]:
from sklearn.linear_model import Lasso
from sklearn.feature_selection import SelectFromModel

In [53]:
feature_sel_model = SelectFromModel(Lasso(alpha=0.005, random_state=0))
feature_sel_model.fit(X_train, y_train)

C:\Users\saraa\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:529: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 399176387081.02966, tolerance: 615098412.8205527
  model = cd_fast.enet_coordinate_descent(


SelectFromModel(estimator=Lasso(alpha=0.005, random_state=0))

In [54]:
X_train  = X_train[selected_feat]
X_test = X_test[selected_feat]

<p><strong>Modeling</strong></p>
<p>We will model (1) Linear regression, (2) Random Forest, and (3) SVR and decide which model is best.</p>

<strong>(1) Linear Regression</strong>

In [55]:
#Fit the model
from sklearn.linear_model import LinearRegression
LinearRegressor = LinearRegression()
LinearRegressor.fit(X_train, y_train)

LinearRegression()

In [56]:
#Calculate the R2 score
from sklearn.metrics import r2_score
y_pred = LinearRegressor.predict(X_test)
Linear_Regression_Score = r2_score(y_test, y_pred)
Linear_Regression_Score

0.8359781242064662

In [57]:
from sklearn.metrics import mean_squared_error
MSE_LR = mean_squared_error(y_test, y_pred, squared = False)
MSE_LR

33831.34682705405

<strong>(2) Random Forest</strong>

In [58]:
from sklearn.ensemble import RandomForestRegressor
RFR = RandomForestRegressor()
RFR.fit(X_train,y_train)

RandomForestRegressor()

In [59]:
y_pred_RFR = RFR.predict(X_test)
RFR_Score = r2_score(y_test, y_pred_RFR)
RFR_Score

0.8915566423936218

In [60]:
MSE_RFR = mean_squared_error(y_test, y_pred_RFR, squared = False)
MSE_RFR

27508.688029936617

<strong>(3) SVR</strong>

In [61]:
from sklearn.svm import SVR
SVReg = SVR(kernel = 'rbf')
SVReg.fit(X_train, y_train)

SVR()

In [62]:
y_pred_SVR = SVReg.predict(X_test)
SVR_Score = r2_score(y_test, y_pred_SVR)
SVR_Score

-0.03120565435172762

In [63]:
MSE_SVR = mean_squared_error(y_test, y_pred_SVR, squared = False)
MSE_SVR

84828.35459253659